In [14]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

In [15]:
# Import the model to a new object
nn_imported = tf.keras.models.load_model('AlphabetSoupCharity.h5')

In [16]:
#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [17]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df=application_df.drop(columns=["NAME","EIN"])

# Determine the number of unique values in each column.
# Generate our categorical variable list

application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
application_df[application_cat].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [18]:
APPLICATION_TYPE=application_df.APPLICATION_TYPE.value_counts()
# Determine which values to replace if counts are less than ...?
replace_application = list(APPLICATION_TYPE[APPLICATION_TYPE < 500].index)
# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [19]:
# Look at CLASSIFICATION value counts for binning
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2380        1
C1732        1
C2570        1
C1728        1
C1580        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [20]:
CLASSIFICATION=application_df.CLASSIFICATION.value_counts()
# Determine which values to replace if counts are less than ..?
replace_class=list(CLASSIFICATION[CLASSIFICATION<700].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [21]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))


# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [22]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [23]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [24]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [25]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 80
hidden_nodes_layer3 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))
# third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               4500      
_________________________________________________________________
dense_1 (Dense)              (None, 80)                8080      
_________________________________________________________________
dense_2 (Dense)              (None, 30)                2430      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 31        
Total params: 15,041
Trainable params: 15,041
Non-trainable params: 0
_________________________________________________________________


In [27]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [29]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"
# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=1000)
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 64016.3545 - accuracy: 0.5080
Epoch 2/100
176/804 [=====>........................] - ETA: 0s - loss: 27148.0404 - accuracy: 0.4620
Epoch 00002: saving model to checkpoints\weights.02.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 25664.3744 - accuracy: 0.4827
Epoch 3/100
380/804 [=============>................] - ETA: 0s - loss: 7134.6300 - accuracy: 0.4968
Epoch 00003: saving model to checkpoints\weights.03.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 10121.7193 - accuracy: 0.4948
Epoch 4/100
562/804 [===================>..........] - ETA: 0s - loss: 37705.0094 - accuracy: 0.5049
Epoch 00004: saving model to checkpoints\weights.04.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 35842.4944 - accuracy: 0.5045
Epoch 5/100
762/804 [===========================>..] - ETA: 0s - loss: 14567.9647 - accuracy: 0.4884
Epoch 00005: saving model to checkpoints\w

Epoch 39/100
421/804 [==============>...............] - ETA: 0s - loss: 0.6921 - accuracy: 0.5251
Epoch 00039: saving model to checkpoints\weights.39.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6918 - accuracy: 0.5271
Epoch 40/100
639/804 [======================>.......] - ETA: 0s - loss: 0.6913 - accuracy: 0.5318
Epoch 00040: saving model to checkpoints\weights.40.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6913 - accuracy: 0.5318
Epoch 41/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6918 - accuracy: 0.5267
Epoch 42/100
  1/804 [..............................] - ETA: 1s - loss: 0.7021 - accuracy: 0.4688
Epoch 00042: saving model to checkpoints\weights.42.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6914 - accuracy: 0.5316
Epoch 43/100
206/804 [======>.......................] - ETA: 0s - loss: 0.6909 - accuracy: 0.5341
Epoch 00043: saving model to checkpoints\weights.43.hdf5
804/804 [==

804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5347
Epoch 77/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6917 - accuracy: 0.5286
Epoch 78/100
 84/804 [==>...........................] - ETA: 0s - loss: 0.6944 - accuracy: 0.5162
Epoch 00078: saving model to checkpoints\weights.78.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6921 - accuracy: 0.5262
Epoch 79/100
264/804 [========>.....................] - ETA: 0s - loss: 0.6896 - accuracy: 0.5433
Epoch 00079: saving model to checkpoints\weights.79.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6907 - accuracy: 0.5358
Epoch 80/100
459/804 [================>.............] - ETA: 0s - loss: 0.6913 - accuracy: 0.5321
Epoch 00080: saving model to checkpoints\weights.80.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6913 - accuracy: 0.5316
Epoch 81/100
670/804 [========================>.....] - ETA: 0s - loss: 0.690

In [30]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6798 - accuracy: 0.6057
Loss: 0.6798194646835327, Accuracy: 0.6057142615318298


In [32]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 100
hidden_nodes_layer3 = 50

nn1 = tf.keras.models.Sequential()

# First hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))
# third hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn1.add(tf.keras.layers.Dense(units=1, activation="tanh"))

# Check the structure of the model
nn1.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 100)               4500      
_________________________________________________________________
dense_9 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_10 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 51        
Total params: 19,701
Trainable params: 19,701
Non-trainable params: 0
_________________________________________________________________


In [38]:
# Compile the model
nn1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"
# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=1000)
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=200,callbacks=[cp_callback])

Epoch 1/200
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 2/200
177/804 [=====>........................] - ETA: 0s - loss: 0.6898 - accuracy: 0.5424
Epoch 00002: saving model to checkpoints\weights.02.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 3/200
367/804 [============>.................] - ETA: 0s - loss: 0.6909 - accuracy: 0.5340
Epoch 00003: saving model to checkpoints\weights.03.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 4/200
548/804 [===================>..........] - ETA: 0s - loss: 0.6913 - accuracy: 0.5314
Epoch 00004: saving model to checkpoints\weights.04.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 5/200
748/804 [==========================>...] - ETA: 0s - loss: 0.6911 - accuracy: 0.5325
Epoch 00005: saving model to checkpoints\weights.05.hdf5
804/804 [=======

426/804 [==============>...............] - ETA: 0s - loss: 0.6911 - accuracy: 0.5326
Epoch 00039: saving model to checkpoints\weights.39.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 40/200
619/804 [======================>.......] - ETA: 0s - loss: 0.6912 - accuracy: 0.5319
Epoch 00040: saving model to checkpoints\weights.40.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 41/200
804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 42/200
  1/804 [..............................] - ETA: 0s - loss: 0.6956 - accuracy: 0.5000
Epoch 00042: saving model to checkpoints\weights.42.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 43/200
210/804 [======>.......................] - ETA: 0s - loss: 0.6913 - accuracy: 0.5308
Epoch 00043: saving model to checkpoints\weights.43.hdf5
804/804 [===============

804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 77/200
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 78/200
 80/804 [=>............................] - ETA: 0s - loss: 0.6928 - accuracy: 0.5188
Epoch 00078: saving model to checkpoints\weights.78.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 79/200
281/804 [=========>....................] - ETA: 0s - loss: 0.6906 - accuracy: 0.5364
Epoch 00079: saving model to checkpoints\weights.79.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 80/200
483/804 [=================>............] - ETA: 0s - loss: 0.6910 - accuracy: 0.5334
Epoch 00080: saving model to checkpoints\weights.80.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 81/200
649/804 [=======================>......] - ETA: 0s - loss: 0.690

804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 115/200
344/804 [===========>..................] - ETA: 0s - loss: 0.6914 - accuracy: 0.5303
Epoch 00115: saving model to checkpoints\weights.115.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 116/200
540/804 [===================>..........] - ETA: 0s - loss: 0.6908 - accuracy: 0.5346
Epoch 00116: saving model to checkpoints\weights.116.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 117/200
724/804 [==========================>...] - ETA: 0s - loss: 0.6910 - accuracy: 0.5329
Epoch 00117: saving model to checkpoints\weights.117.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 118/200
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 119/200
122/804 [===>..........................] - ETA: 0s - los

557/804 [===================>..........] - ETA: 0s - loss: 0.6912 - accuracy: 0.5315
Epoch 00152: saving model to checkpoints\weights.152.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 153/200
771/804 [===========================>..] - ETA: 0s - loss: 0.6911 - accuracy: 0.5320
Epoch 00153: saving model to checkpoints\weights.153.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 154/200
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 155/200
159/804 [====>.........................] - ETA: 0s - loss: 0.6911 - accuracy: 0.5326
Epoch 00155: saving model to checkpoints\weights.155.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 156/200
376/804 [=============>................] - ETA: 0s - loss: 0.6910 - accuracy: 0.5330
Epoch 00156: saving model to checkpoints\weights.156.hdf5
804/804 [=======

 42/804 [>.............................] - ETA: 0s - loss: 0.6889 - accuracy: 0.5506
Epoch 00190: saving model to checkpoints\weights.190.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 191/200
208/804 [======>.......................] - ETA: 0s - loss: 0.6899 - accuracy: 0.5422
Epoch 00191: saving model to checkpoints\weights.191.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 192/200
432/804 [===============>..............] - ETA: 0s - loss: 0.6910 - accuracy: 0.5328
Epoch 00192: saving model to checkpoints\weights.192.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 193/200
600/804 [=====================>........] - ETA: 0s - loss: 0.6914 - accuracy: 0.5302
Epoch 00193: saving model to checkpoints\weights.193.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 194/200
804/804 [=======

In [39]:
# Evaluate the second model using the test data
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6927 - accuracy: 0.5673
Loss: 0.6926688551902771, Accuracy: 0.5673469305038452


In [41]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 80
hidden_nodes_layer3 = 50

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))
# third hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 100)               4500      
_________________________________________________________________
dense_19 (Dense)             (None, 80)                8080      
_________________________________________________________________
dense_20 (Dense)             (None, 50)                4050      
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 51        
Total params: 16,681
Trainable params: 16,681
Non-trainable params: 0
_________________________________________________________________


In [44]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"
# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=1000)
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=200,callbacks=[cp_callback])

Epoch 1/200
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 2/200
193/804 [======>.......................] - ETA: 0s - loss: 0.6901 - accuracy: 0.5400
Epoch 00002: saving model to checkpoints\weights.02.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 3/200
364/804 [============>.................] - ETA: 0s - loss: 0.6916 - accuracy: 0.5282
Epoch 00003: saving model to checkpoints\weights.03.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 4/200
572/804 [====================>.........] - ETA: 0s - loss: 0.6913 - accuracy: 0.5303
Epoch 00004: saving model to checkpoints\weights.04.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 5/200
780/804 [============================>.] - ETA: 0s - loss: 0.6911 - accuracy: 0.5319
Epoch 00005: saving model to checkpoints\weights.05.hdf5
804/804 [=======

433/804 [===============>..............] - ETA: 0s - loss: 0.6917 - accuracy: 0.5275
Epoch 00039: saving model to checkpoints\weights.39.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 40/200
616/804 [=====================>........] - ETA: 0s - loss: 0.6916 - accuracy: 0.5282
Epoch 00040: saving model to checkpoints\weights.40.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 41/200
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 42/200
  1/804 [..............................] - ETA: 1s - loss: 0.6912 - accuracy: 0.5312
Epoch 00042: saving model to checkpoints\weights.42.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 43/200
206/804 [======>.......................] - ETA: 0s - loss: 0.6909 - accuracy: 0.5340
Epoch 00043: saving model to checkpoints\weights.43.hdf5
804/804 [===============

804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 77/200
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 78/200
 79/804 [=>............................] - ETA: 0s - loss: 0.6914 - accuracy: 0.5305
Epoch 00078: saving model to checkpoints\weights.78.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 79/200
267/804 [========>.....................] - ETA: 0s - loss: 0.6915 - accuracy: 0.5294
Epoch 00079: saving model to checkpoints\weights.79.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 80/200
478/804 [================>.............] - ETA: 0s - loss: 0.6910 - accuracy: 0.5329
Epoch 00080: saving model to checkpoints\weights.80.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 81/200
680/804 [========================>.....] - ETA: 0s - loss: 0.691

804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 115/200
330/804 [===========>..................] - ETA: 0s - loss: 0.6908 - accuracy: 0.5346
Epoch 00115: saving model to checkpoints\weights.115.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 116/200
532/804 [==================>...........] - ETA: 0s - loss: 0.6909 - accuracy: 0.5335 ETA: 0s - loss: 0.6910 - accuracy
Epoch 00116: saving model to checkpoints\weights.116.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 117/200
706/804 [=========================>....] - ETA: 0s - loss: 0.6909 - accuracy: 0.5336
Epoch 00117: saving model to checkpoints\weights.117.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 118/200
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 119/200
113/804 [===>.........

804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 152/200
564/804 [====================>.........] - ETA: 0s - loss: 0.6911 - accuracy: 0.5326
Epoch 00152: saving model to checkpoints\weights.152.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 153/200
773/804 [===========================>..] - ETA: 0s - loss: 0.6909 - accuracy: 0.5336
Epoch 00153: saving model to checkpoints\weights.153.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 154/200
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 155/200
162/804 [=====>........................] - ETA: 0s - loss: 0.6892 - accuracy: 0.5459
Epoch 00155: saving model to checkpoints\weights.155.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 156/200
365/804 [============>.................] - ETA: 0s - los

804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 189/200
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 190/200
 39/804 [>.............................] - ETA: 1s - loss: 0.6937 - accuracy: 0.5104
Epoch 00190: saving model to checkpoints\weights.190.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 191/200
203/804 [======>.......................] - ETA: 0s - loss: 0.6916 - accuracy: 0.5285
Epoch 00191: saving model to checkpoints\weights.191.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 192/200
403/804 [==============>...............] - ETA: 0s - loss: 0.6913 - accuracy: 0.5303
Epoch 00192: saving model to checkpoints\weights.192.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 193/200
608/804 [=====================>........] - ETA: 0s - los

In [45]:
# Evaluate the second model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 6.4964 - accuracy: 0.4668
Loss: 6.496426105499268, Accuracy: 0.46682214736938477


In [47]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest predictive accuracy: 0.710


In [ ]:
# it turns out that my first nerual network model nn has the highest accruacy score: about 60%
# output nn as my optimazation model
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")